In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import json 

session = requests.session()

# Scrap item lists: players and teams tags

In [145]:
URL = 'https://www.handball-base.com/players'
players_list = []
p = re.compile('<a class="item playerWrapper" href="(.*?)">')  # ej: <a class="item playerWrapper" href="elia-steffen">

for pages in range(1, 237):
    req = requests.post(URL, params={'p': pages})
    soup = BeautifulSoup(req.content, 'html.parser')
    results = soup.find_all("a", {"class": "item playerWrapper"})

    for player in range(len(results)):
        data  = str(results[player])
        m = p.match(data)
        players_list.append(m[1])

print(len(players_list))
with open('./tags_players.json', 'w', encoding='utf-8') as f:
    json.dump(players_list, f, ensure_ascii=False, indent=4)

9280


In [81]:
URL = 'https://www.handball-base.com/clubs'
clubs_list = []
p = re.compile('<a class="item clubWrapper" href="(.*?)">')  # ej: <a href="sydney-uni-handball-club" class="item clubWrapper">

for pages in range(1, 26):
    req = requests.post(URL, params={'p': pages})
    soup = BeautifulSoup(req.content, 'html.parser')
    results = soup.find_all("a", {"class": "item clubWrapper"})

    for clubs in range(len(results)):
        data  = str(results[clubs])
        m = p.match(data)
        clubs_list.append(m[1])

print(len(clubs_list))
with open('./tags_clubs.json', 'w', encoding='utf-8') as f:
    json.dump(clubs_list, f, ensure_ascii=False, indent=4)

972


# Scrap each player

In [195]:
URL = 'https://www.handball-base.com/arenas'
arena_list = []
p = re.compile('<a class="arenaWrapper" href="(.*?)">')  # ej: <a href="arena-du-pays-d-aix" class="arenaWrapper">

for pages in range(1, 6):
    req = requests.post(URL, params={'p': pages})
    soup = BeautifulSoup(req.content, 'html.parser')
    results = soup.find_all("a", {"class": "arenaWrapper"})

    for arenas in range(len(results)):
        areanas_dict = {}
        data  = str(results[arenas])
        m = p.match(data)
        name = soup.select(".arenaWrapper .textWrapper .title")[arenas*2].get_text()
        location = soup.select(".arenaWrapper .textWrapper .location .title")[arenas].get_text().split(', ')

        areanas_dict['name'] = name
        areanas_dict['arena_search'] = m[1]
        areanas_dict['country'] = location[1]
        areanas_dict['city'] = location[0]

        arena_list.append(areanas_dict)

print(len(arena_list))
with open('./data_arenas.json', 'w', encoding='utf-8') as f:
    json.dump(arena_list, f, ensure_ascii=False, indent=4)

df_arenas = pd.DataFrame(arena_list)
df_arenas.to_csv('./data_arenas.csv')

198


In [151]:
# JSON file
f = open ('./data.json', "r")
players_list = json.loads(f.read())
print(len(players_list))
print(players_list[:5])

9280
['ali-kouhzad', 'shahab-sadeghzadeh', 'omid-reza-sarpoushi', 'afshin-sadeghi', 'mehrdad-samsami']


In [170]:
all_players = []
var_seguimiento = 0

for player in players_list:

    var_seguimiento=var_seguimiento+1
    if var_seguimiento%500 == 0:
        print(var_seguimiento)

    URL = 'https://www.handball-base.com/'+player
    req = requests.post(URL)
    soup = BeautifulSoup(req.content, 'html.parser')
    player_dict = {}
    
    if "404 Error" not in str(soup):

        if len(soup.select(".mainInfoWrapper")) != 0:
            name = str(soup.select(".mainInfoWrapper > h1")[0]).replace("<h1>","")
            name = name.replace("</h1>","")
            player_dict['name'] = name
            player_dict['name_search'] = player

            p_label = re.compile('<span class="label">(.*?)</span>') 
            p_data = re.compile('<span class="data">(.*?)</span>') 
            p_data_position = re.compile('<span class="data">\n<span>\n\t\t\t\t\t\t\t\t\t(.*?)\t\t\t\t\t\t\t\t</span></span>') 
            p_data_active = re.compile('<span class="data green">(.*?)</span>')
            p_data_retired = re.compile('<span class="data red">(.*?)</span>')
            player_label = soup.select(".clubInfo > p > .label") # <span class="label">Nationality</span>
            player_data = soup.select(".clubInfo > p > .data") # <span class="data">Switzerland</span>

            for d in range(len(player_label)):
                value_label = p_label.match(str(player_label[d]))[1]
                if value_label == 'Position':
                    data_value = p_data_position.match(str(player_data[d]))[1]
                elif value_label == 'Status':
                    if "data green" in str(player_data[d]):
                        data_value = p_data_active.match(str(player_data[d]))[1]
                    else:
                        data_value = p_data_retired.match(str(player_data[d]))[1]
                else:
                    data_value = p_data.match(str(player_data[d]))[1]
                value_label = value_label.lower()
                player_dict[value_label] = data_value

            club_name = soup.select(".currentClubInfo .title a")[0].get_text()
            club_name_search = re.search("href=(.*?)>", str(soup.select(".currentClubInfo .title a")[0]))
            player_dict['club'] = club_name
            player_dict['club_search'] = club_name_search.group(1)

            for c in range(1, 1+len(soup.select(".currentClubInfo p")[1:])):
                club_text = re.search("<p>(.*?): (.*?)</p>", str(soup.select(".currentClubInfo p")[c]))
                club_label = club_text.group(1).lower()
                club_data = club_text.group(2)
                player_dict[club_label] = club_data

            all_players.append(player_dict)
        else: 
            player_dict['name_search'] = player
            all_players.append(player_dict)
    else:
        player_dict['name'] = "error"
        player_dict['name_search'] = player
        all_players.append(player_dict)
    

with open('./data_players.json', 'w', encoding='utf-8') as f:
    json.dump(all_players, f, ensure_ascii=False, indent=4)

df_players = pd.DataFrame(all_players)
df_players.to_csv('./data_players.csv')

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500


TypeError: 'NoneType' object is not subscriptable

In [94]:
# load clubs JSON file
f = open ('./tags_clubs.json', "r")
clubs_list = json.loads(f.read())
print(len(clubs_list))
print(clubs_list[:5])

972
['sk-hawks', 'doosan', 'incheon-city', 'chungnam', 'sangmu-phoenix']


In [112]:
all_clubs = []
var_seguimiento = 0

for club in clubs_list:

    var_seguimiento=var_seguimiento+1
    if var_seguimiento%200 == 0:
        print(var_seguimiento)

    URL = 'https://www.handball-base.com/'+club
    req = requests.post(URL)
    soup = BeautifulSoup(req.content, 'html.parser')
    club_dict = {}

    if "404 Error" not in str(soup):

        if len(soup.select(".mainInfoWrapper")) != 0:
            name = str(soup.select(".mainInfoWrapper > h1")[0].get_text())
            club_dict['name'] = name
            club_dict['name_search'] = club

            p_label = re.compile('<span class="label">(.*?)</span>') 
            p_data = re.compile('<span class="data">(.*?)</span>') 
            club_label = soup.select(".clubInfo > p > .label") # <span class="label">Nationality</span>
            club_data = soup.select(".clubInfo > p > .data") # <span class="data">Switzerland</span>

            for d in range(len(club_label)):
                value_label = p_label.match(str(club_label[d]))[1]
                data_value = p_data.match(str(club_data[d]))[1]
                value_label = value_label.lower()
                club_dict[value_label] = data_value
            
            arena = str(soup.select(".currentClubInfo .title a")[0].get_text())
            club_dict["arena"] = arena

            all_clubs.append(club_dict)

        # if there's no data in response, save param
        else: 
            club_dict['name_search'] = club
            all_clubs.append(club_dict)

print(len(all_clubs))
with open('./data_clubs.json', 'w', encoding='utf-8') as f:
    json.dump(all_clubs, f, ensure_ascii=False, indent=4)

df_clubs = pd.DataFrame(all_clubs)
df_clubs.to_csv('./data_clubs.csv')

200
400
600
800
970


# Clean Data

In [ ]:
# load clubs csv file
df_players = pd.read_csv('./data_players.csv')
df_players.head()

In [189]:
all_players = []
var_seguimiento = 0
print("jugadores no encontrados:", len(df_players.loc[df_players.name.isnull(), 'name_search']))

for player in list(df_players.loc[df_players.name.isnull(), 'name_search']):

    URL = 'https://www.handball-base.com/'+player
    req = requests.post(URL)
    soup = BeautifulSoup(req.content, 'html.parser')
    
    if len(soup.select(".mainInfoWrapper")) != 0:
        name = str(soup.select(".mainInfoWrapper > h1")[0].get_text())
        df_players.loc[df_players.name_search == player, "name"] = name

        p_label = re.compile('<span class="label">(.*?)</span>') 
        p_data = re.compile('<span class="data">(.*?)</span>') 
        p_data_position = re.compile('<span class="data">\n<span>\n\t\t\t\t\t\t\t\t\t(.*?)\t\t\t\t\t\t\t\t</span></span>') 
        p_data_active = re.compile('<span class="data green">(.*?)</span>')
        p_data_retired = re.compile('<span class="data red">(.*?)</span>')
        player_label = soup.select(".clubInfo > p > .label") # <span class="label">Nationality</span>
        player_data = soup.select(".clubInfo > p > .data") # <span class="data">Switzerland</span>

        for d in range(len(player_label)):
            value_label = p_label.match(str(player_label[d]))[1]
            if value_label == 'Position':
                data_value = p_data_position.match(str(player_data[d]))[1]
            elif value_label == 'Status':
                if "data green" in str(player_data[d]):
                    data_value = p_data_active.match(str(player_data[d]))[1]
                else:
                    data_value = p_data_retired.match(str(player_data[d]))[1]
            else:
                data_value = p_data.match(str(player_data[d]))[1]
            value_label = value_label.lower()
            df_players.loc[df_players.name_search == player, value_label] = data_value

        club_name = soup.select(".currentClubInfo .title a")[0].get_text()
        club_name_search = re.search("href=(.*?)>", str(soup.select(".currentClubInfo .title a")[0]))
        df_players.loc[df_players.name_search == player, "club"] = club_name
        df_players.loc[df_players.name_search == player, "club_search"] = club_name_search.group(1)

        for c in range(1, 1+len(soup.select(".currentClubInfo p")[1:])):
            club_text = re.search("<p>(.*?): (.*?)</p>", str(soup.select(".currentClubInfo p")[c]))
            club_label = club_text.group(1).lower()
            club_data = club_text.group(2)
            df_players.loc[df_players.name_search == player, club_label] = club_data
            player_dict[club_label] = club_data

print("jugadores no corregidos:", len(df_players.loc[df_players.name.isnull(), 'name_search']))

jugadores no encontrados: 36
jugadores no corregidos: 0


In [192]:
print(list(df_players.loc[df_players.name == 'error', 'name_search']))
df_players = df_players.loc[df_players.name != 'error'].reset_index(drop=True)

['david-vecko', 'david-vecko', 'david-vecko', 'david-vecko', 'jovan-djuriÄ‡1', 'jovan-djuriÄ‡1']


In [193]:
df_players.to_csv('./data_players_v2.csv')

In [138]:
# load clubs csv file
df_clubs = pd.read_csv('./data_clubs.csv')
df_clubs.head()

,Unnamed: 0,name,name_search,country,city,league,founding year,arena,web site
0,0,SK Hawks,sk-hawks,South Korea,Cheongju,Handball Korea League (South Korea),No info,NaN,NaN
1,1,Doosan,doosan,South Korea,South Gyeongsang Province,Handball Korea League (South Korea),1991,NaN,NaN
2,2,Incheon City,incheon-city,South Korea,Incheon,Handball Korea League (South Korea),2006,NaN,NaN
3,3,Chungnam,chungnam,South Korea,Chungnam,Handball Korea League (South Korea),No info,NaN,NaN
4,4,Sangmu Phoenix,sangmu-phoenix,South Korea,Mungyeong,Handball Korea League (South Korea),No info,NaN,NaN


In [144]:
for club in list(df_clubs.loc[df_clubs.name.isnull(), 'name_search']):

    URL = 'https://www.handball-base.com/'+club
    req = requests.post(URL)
    soup = BeautifulSoup(req.content, 'html.parser')

    if "404 Error" not in str(soup):
        club_dict = {}

        if len(soup.select(".mainInfoWrapper")) != 0:
            name = str(soup.select(".mainInfoWrapper > h1")[0].get_text())
            df_clubs.loc[df_clubs.name_search == club, "name"] = name

            p_label = re.compile('<span class="label">(.*?)</span>') 
            p_data = re.compile('<span class="data">(.*?)</span>') 
            club_label = soup.select(".clubInfo > p > .label") # <span class="label">Nationality</span>
            club_data = soup.select(".clubInfo > p > .data") # <span class="data">Switzerland</span>

            for d in range(len(club_label)):
                value_label = p_label.match(str(club_label[d]))[1]
                data_value = p_data.match(str(club_data[d]))[1]
                value_label = value_label.lower()
                df_clubs.loc[df_clubs.name_search == club, value_label] = data_value
            
            arena = str(soup.select(".currentClubInfo .title a")[0].get_text())
            df_clubs.loc[df_clubs.name_search == club, "arena"] = arena


        # if there's no data in response, save param
        else: 
            print("no conseguido", club)

In [197]:
df_players

,name,name_search,nationality,birth date,height,position,status,club,club_search,country,league,weight
0,Ali Kouhzad,ali-kouhzad,Iran,19.12.1996.,198 cm,Line Player,active,SS Kazeroon Club,"""ss-kazeroon-club""",Iran,Iran Premier League,NaN
1,Shahab Sadeghzadeh,shahab-sadeghzadeh,Iran,28.03.1994.,192 cm,Left Back,active,Sepahan,"""sepahan""",Iran,Iran Premier League,NaN
2,Omid Reza Sarpoushi,omid-reza-sarpoushi,Iran,09.03.1998.,190 cm,Goalkeeper,active,Mes Kerman,"""mes-kerman""",Iran,Iran Premier League,NaN
3,Afshin Sadeghi,afshin-sadeghi,Iran,25.03.1993.,198 cm,Left Back,active,Sepahan,"""sepahan""",Iran,Iran Premier League,NaN
4,Mehrdad Samsami,mehrdad-samsami,Iran,26.02.1991.,194 cm,Centre Back,active,SS Kazeroon Club,"""ss-kazeroon-club""",Iran,Iran Premier League,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5830,József Albek,jozsef-albek,Hungary,30.06.1999.,195 cm,Left Back,active,HSG Graz,"""hsg-graz""",Austria,SPUSU Liga (Austria),NaN
5831,Daniel Dicker,daniel-dicker,Austria,05.06.1995.,199 cm,Left Back,active,UHK Krems,"""uhk-krems""",Austria,SPUSU Liga (Austria),NaN
5832,Nemanja Beloš,nemanja-belos,Serbia,08.06.1994.,192 cm,Left Back,active,HSG Graz,"""hsg-graz""",Austria,SPUSU Liga (Austria),NaN
5833,Christian Hallmann,christian-hallmann,Austria,21.12.1992.,185 cm,Line Player,active,HSG Graz,"""hsg-graz""",Austria,SPUSU Liga (Austria),NaN
